# Code for calculating the final dataframes for regression of the effect of key moments on the flow of momentum. 

In [ ]:
#find the maximum and minimum values for pc and pcv used for normalizing the momentum values
import os
from progressbar import ProgressBar
import pandas as pd
import pickle

#create list with all needed files
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []
for file in os.listdir(path):
    if 'pc_over_last_two_minutes_final' in file:
        pc_matches.append(file[30:37])

#create empty dataframe start iterating over each pitch control match
pbar = ProgressBar()
maximum = -1000
minimum = 1000

for game in pbar(pc_matches):
    #open pitch control file
    infile = open('pc_over_last_two_minutes_final' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    #subtract pcv values so that both teams cannot have momentum at the same time
    df_pc['pcv home'] = df_pc['pcv home'] - df_pc['pcv away']
    df_pc['pcv away'] = df_pc['pcv away'] - df_pc['pcv home']
    #find minimum and maximum
    maximum_pcv = max(maximum, df_pc['pcv home'].max(), df_pc['pcv away'].max())
    minimum_pcv = min(minimum, df_pc['pcv home'].min(), df_pc['pcv away'].min())
    maximum_pc = max(maximum, df_pc['pc home'].max(), df_pc['pc away'].max())
    minimum_pc = min(minimum, df_pc['pc home'].min(), df_pc['pc away'].min())
    
    

In [ ]:
#Calculations that will be used for regression input for key moments on the flow of momentum (PC anc PCV calculations)
import os
from progressbar import ProgressBar
import pandas as pd
import pickle

#create list with all pitch contol files including two-minute sliding time windows
path = '/Users/gebruiker/Dropbox/Master Thesis/Master thesis RKC Waalwijk/Code'
pc_matches = []
for file in os.listdir(path):
    if 'pc_over_last_two_minutes_final' in file:
        pc_matches.append(file[30:37])

#create empty dataframe start iterating over each pitch control match
final_df = pd.DataFrame()
pbar = ProgressBar()
for game in pbar(pc_matches):
    #open pitch control file
    infile = open('pc_over_last_two_minutes_final' + game + '.p','rb')
    df_pc = pickle.load(infile)
    infile.close()
    #subtract the pcv values so that both teams cannot have momentum at the same time and normalize the values
    df_pc['momentum pcv home previous 2 minutes'] = df_pc['momentum pcv home previous 2 minutes'] - df_pc['momentum pcv away previous 2 minutes']
    df_pc['momentum pcv away previous 2 minutes'] = df_pc['momentum pcv away previous 2 minutes'] - df_pc['momentum pcv home previous 2 minutes']
    df_pc['pcv home'] = df_pc['pcv home'] - df_pc['pcv away']
    df_pc['pcv away'] = df_pc['pcv away'] - df_pc['pcv home']
    df_pc['momentum pcv home previous 2 minutes'] = (df_pc['momentum pcv home previous 2 minutes']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['momentum pcv away previous 2 minutes'] = (df_pc['momentum pcv away previous 2 minutes']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['pcv home'] = (df_pc['pcv home']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['pcv away'] = (df_pc['pcv away']-minimum_pcv)/(maximum_pcv-minimum_pcv)
    df_pc['momentum pc home previous 2 minutes'] = (df_pc['momentum pc home previous 2 minutes']-minimum_pc)/(maximum_pc-minimum_pc)
    df_pc['momentum pc away previous 2 minutes'] = (df_pc['momentum pc away previous 2 minutes']-minimum_pc)/(maximum_pc-minimum_pc)
    df_pc['pc home'] = (df_pc['pc home']-minimum_pc)/(maximum_pc-minimum_pc)
    df_pc['pc away'] = (df_pc['pc away']-minimum_pc)/(maximum_pc-minimum_pc)
    
    #import the shots and goals dataframe
    infile = open('shots and goals event dataframes.p','rb')
    df_shots = pickle.load(infile)
    infile.close()
    
    #filter the shots and goals dataframe on the current match
    df_shots = df_shots.loc[df_shots['match_id'] == game]
    
    #for each shot and goals: add whether it is favorable/unfavorable for the home team
    type_key = []
    key_mom = []
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['team'] == 'Home':
            key_mom.append('favorable')
            type_key.append(1)
        else:
            key_mom.append('unfavorable')
            type_key.append(-1)

    df_shots['key_moment'] = key_mom
    df_shots['type_key'] = type_key

    #select only the necessary columns
    df_shots = df_shots[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'outcome', 'match_id', 'key_moment', 'type_key']]

    #filter based on the first and second half
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    fh = df_pc.loc[df_pc['Half'] == 1]
    sh = df_pc.loc[df_pc['Half'] == 2]
    
    #initialize lists for the vairables in the regression model
    pcbhome = []
    pcbaway = []
    pcvbhome = []
    pcvbaway = []
    pcahome = []
    pcaaway = []
    pcvahome = []
    pcvaaway = []
    deltamompchome = []
    deltamompcaway = []
    deltamompcvhome = []
    deltamompcvaway = []
    deltapchome = []
    deltapcaway = []
    deltapcvhome = []
    deltapcvaway = []
    varpchome = []
    varpcaway = []
    varpcvhome = []
    varpcvaway = []
    index_min_below = []
    index_min_above = []
    
    #for each shot or goal in the first half: add the home and away values for the necessary variables
    for i in range(len(df_shots_fh)):
        minute = float(df_shots_fh.iloc[i]['minute'])
        second = float(df_shots_fh.iloc[i]['seconds'])
        #find the indices in the PC and PCV dataset corresponding to the time of the shot or goal
        try:
            index_above = fh[fh['Time'] > float((minute) + second/60)].iloc[0].name
        except:
            index_above = fh[fh['Time'] < float((minute) + second/60)].iloc[-1].name
        try:
            index_below = fh[fh['Time'] <= float((minute) + second/60)].iloc[-1].name
        except:
            index_below = index_above
        
        #select the 2-mnute timeframes for which the variables will be added
        index_2min_above = fh.index[fh.index <= index_above + 2*60*25].max()
        index_4min_above = fh.index[fh.index <= index_above + 4*60*25].max()
        index_2min_below = fh.index[fh.index >= index_below - 2*60*25].min()
        df_i_pc_b = fh.loc[index_2min_below:index_below]
        df_i_pc_a = fh.loc[index_2min_above:index_4min_above]
        #add the necessary variables
        pcbhome.append(df_i_pc_b['momentum pc home previous 2 minutes'].mean())
        pcbaway.append(df_i_pc_b['momentum pc away previous 2 minutes'].mean())
        pcvbhome.append(df_i_pc_b['momentum pcv home previous 2 minutes'].mean())
        pcvbaway.append(df_i_pc_b['momentum pcv away previous 2 minutes'].mean())
        pcahome.append(df_i_pc_a['momentum pc home previous 2 minutes'].mean())
        pcaaway.append(df_i_pc_a['momentum pc away previous 2 minutes'].mean())
        pcvahome.append(df_i_pc_a['momentum pcv home previous 2 minutes'].mean())
        pcvaaway.append(df_i_pc_a['momentum pcv away previous 2 minutes'].mean())
        deltamompchome.append(pcahome[-1] - pcbhome[-1])
        deltamompcaway.append(pcaaway[-1] - pcbaway[-1])
        deltamompcvhome.append(pcvahome[-1] - pcvbhome[-1])
        deltamompcvaway.append(pcvaaway[-1] - pcvbaway[-1])
        deltapchome.append(df_i_pc_b.iloc[-1]['pc home'] - df_i_pc_b.iloc[0]['pc home'])
        deltapcaway.append(df_i_pc_b.iloc[-1]['pc away'] - df_i_pc_b.iloc[0]['pc away'])
        deltapcvhome.append(df_i_pc_b.iloc[-1]['pcv home'] - df_i_pc_b.iloc[0]['pcv home'])
        deltapcvaway.append(df_i_pc_b.iloc[-1]['pcv away'] - df_i_pc_b.iloc[0]['pcv away'])
        varpchome.append(sum(abs(df_i_pc_b['pc home'].diff()[1:])))
        varpcaway.append(sum(abs(df_i_pc_b['pc away'].diff()[1:])))
        varpcvhome.append(sum(abs(df_i_pc_b['pcv home'].diff()[1:])))
        varpcvaway.append(sum(abs(df_i_pc_b['pcv away'].diff()[1:])))
        index_min_below.append(index_2min_below)
        index_min_above.append(index_2min_above)
        
    #do the same for the second half
    for i in range(len(df_shots_sh)):
        minute = float(df_shots_sh.iloc[i]['minute'])
        second = float(df_shots_sh.iloc[i]['seconds'])
        try:
            index_above = sh[sh['Time'] > float((minute-45) + second/60)].iloc[0].name
        except:
            index_above = sh[sh['Time'] < float((minute-45) + second/60)].iloc[-1].name
        try:
            index_below = sh[sh['Time'] <= float((minute-45) + second/60)].iloc[-1].name
        except:
            index_below = index_above
        
        index_2min_above = sh.index[sh.index <= index_above + 2*60*25].max()
        index_4min_above = sh.index[sh.index <= index_above + 4*60*25].max()
        index_2min_below = sh.index[sh.index >= index_below - 2*60*25].min()
        df_i_pc_b = sh.loc[index_2min_below:index_below]
        df_i_pc_a = sh.loc[index_2min_above:index_4min_above]
        pcbhome.append(df_i_pc_b['momentum pc home previous 2 minutes'].mean())
        pcbaway.append(df_i_pc_b['momentum pc away previous 2 minutes'].mean())
        pcvbhome.append(df_i_pc_b['momentum pcv home previous 2 minutes'].mean())
        pcvbaway.append(df_i_pc_b['momentum pcv away previous 2 minutes'].mean())
        pcahome.append(df_i_pc_a['momentum pc home previous 2 minutes'].mean())
        pcaaway.append(df_i_pc_a['momentum pc away previous 2 minutes'].mean())
        pcvahome.append(df_i_pc_a['momentum pcv home previous 2 minutes'].mean())
        pcvaaway.append(df_i_pc_a['momentum pcv away previous 2 minutes'].mean())
        deltamompchome.append(pcahome[-1] - pcbhome[-1])
        deltamompcaway.append(pcaaway[-1] - pcbaway[-1])
        deltamompcvhome.append(pcvahome[-1] - pcvbhome[-1])
        deltamompcvaway.append(pcvaaway[-1] - pcvbaway[-1])
        deltapchome.append(df_i_pc_b.iloc[-1]['pc home'] - df_i_pc_b.iloc[0]['pc home'])
        deltapcaway.append(df_i_pc_b.iloc[-1]['pc away'] - df_i_pc_b.iloc[0]['pc away'])
        deltapcvhome.append(df_i_pc_b.iloc[-1]['pcv home'] - df_i_pc_b.iloc[0]['pcv home'])
        deltapcvaway.append(df_i_pc_b.iloc[-1]['pcv away'] - df_i_pc_b.iloc[0]['pcv away'])
        varpchome.append(sum(abs(df_i_pc_b['pc home'].diff()[1:])))
        varpcaway.append(sum(abs(df_i_pc_b['pc away'].diff()[1:])))
        varpcvhome.append(sum(abs(df_i_pc_b['pcv home'].diff()[1:])))
        varpcvaway.append(sum(abs(df_i_pc_b['pcv away'].diff()[1:])))
        index_min_below.append(index_2min_below)
        index_min_above.append(index_2min_above)
        
    #add the variables to shots dataframe
    df_shots['mean pc home 2min before'] = pcbhome
    df_shots['mean pc away 2min before'] = pcbaway
    df_shots['mean pc home 2min after'] = pcahome
    df_shots['mean pc away 2min after'] = pcaaway
    df_shots['mean pcv home 2min before'] = pcvbhome
    df_shots['mean pcv away 2min before'] = pcvbaway
    df_shots['mean pcv home 2min after'] = pcvahome
    df_shots['mean pcv away 2min after'] = pcvaaway
    df_shots['delta pc momentum home'] = deltamompchome
    df_shots['delta pc momentum away'] = deltamompcaway
    df_shots['delta pcv momentum home'] = deltamompcvhome
    df_shots['delta pcv momentum away'] = deltamompcvaway
    df_shots['end value pc home'] = endpchome
    df_shots['end value pc away'] = endpcaway
    df_shots['end value pcv home'] = endpcvhome
    df_shots['end value pcv away'] = endpcvaway
    df_shots['delta pc home'] = deltapchome
    df_shots['delta pc away'] = deltapcaway
    df_shots['delta pcv home'] = deltapcvhome
    df_shots['delta pcv away'] = deltapcvaway
    df_shots['var pc home'] = varpchome
    df_shots['var pc away'] = varpcaway
    df_shots['var pcv home'] = varpcvhome
    df_shots['var pcv away'] = varpcvaway
    df_shots['index 2min before'] = index_min_below
    df_shots['index 2min after'] = index_min_above

    #find non-overlapping time windows in which no key moment occurs
    minutes_first_half = []
    minutes_second_half = []
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    
    #first half calculations
    for i in range(len(df_shots_fh)):
        try:
            if i == 0:
                #for the first event: check whether enough time has elapsed to find a time window in which no key moment occurs
                if df_shots_fh.iloc[i]['index 2min before'] - fh.iloc[0].name > 8*60*25:
                    dif = df_shots_fh.iloc[i]['index 2min before'] - fh.iloc[0].name
                    if dif < 12*60*25:
                        minutes_first_half.append(fh.loc[fh.index[fh.index <= fh.iloc[0].name + int(dif/2)].max()]['Time'])
                    elif dif >= 12*60*25 and dif <16*60*25:
                        minutes_first_half.append(fh.loc[fh.index[fh.index <= fh.iloc[0].name + int(dif/3)].max()]['Time'])
                        minutes_first_half.append(fh.loc[fh.index[fh.index <= fh.iloc[0].name + int((dif/3)*2)].max()]['Time'])
                    elif dif >= 16*60*25:
                        minutes_first_half.append(fh.loc[fh.index[fh.index <= fh.iloc[0].name + int(dif/4)].max()]['Time'])
                        minutes_first_half.append(fh.loc[fh.index[fh.index <= fh.iloc[0].name + int((dif/4)*2)].max()]['Time'])
                        minutes_first_half.append(fh.loc[fh.index[fh.index <= fh.iloc[0].name + int((dif/4)*3)].max()]['Time'])

            if df_shots_fh.iloc[i+1]['index 2min before'] - df_shots_fh.iloc[i]['index 2min after'] > 8*60*25:
                #for each event: check whether the time between the this event and the next event is enough to find a time window in which no key moments occurs
                dif = df_shots_fh.iloc[i+1]['index 2min before'] - df_shots_fh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int(dif/2)].max()]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int(dif/3)].max()]['Time'])
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int((dif/3)*2)].max()]['Time'])
                elif dif >= 16*60*25:
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int(dif/4)].max()]['Time'])
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*2)].max()]['Time'])
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*3)].max()]['Time'])

        except:
            if fh.iloc[-1].name - df_shots_fh.iloc[i]['index 2min after'] > 8*60*25:
                #for the last event: check whether there is enough time left to find a key moments in which no key moments occurs
                dif = fh.iloc[-1].name - df_shots_fh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int(dif/2)].max()]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int(dif/3)].max()]['Time'])
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int((dif/3)*2)].max()]['Time'])
                elif dif >= 16*60*25:
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int(dif/4)].max()]['Time'])
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*2)].max()]['Time'])
                    minutes_first_half.append(fh.loc[fh.index[fh.index <= df_shots_fh.iloc[i]['index 2min after'] + int((dif/4)*3)].max()]['Time'])

    #do the same for the second half
    for i in range(len(df_shots_sh)):
        try:
            if i == 0:
                if df_shots_sh.iloc[i]['index 2min before'] - sh.iloc[0].name > 8*60*25:
                    dif = df_shots_sh.iloc[i]['index 2min before'] - sh.iloc[0].name
                    if dif < 12*60*25:
                        minutes_second_half.append(sh.loc[sh.index[sh.index <= sh.iloc[0].name + int(dif/2)].max()]['Time'])
                    elif dif >= 12*60*25 and dif <16*60*25:
                        minutes_second_half.append(sh.loc[sh.index[sh.index <= sh.iloc[0].name + int(dif/3)].max()]['Time'])
                        minutes_second_half.append(sh.loc[sh.index[sh.index <= sh.iloc[0].name + int((dif/3)*2)].max()]['Time'])
                    elif dif >= 16*60*25:
                        minutes_second_half.append(sh.loc[sh.index[sh.index <= sh.iloc[0].name + int(dif/4)].max()]['Time'])
                        minutes_second_half.append(sh.loc[sh.index[sh.index <= sh.iloc[0].name + int((dif/4)*2)].max()]['Time'])
                        minutes_second_half.append(sh.loc[sh.index[sh.index <= sh.iloc[0].name + int((dif/4)*3)].max()]['Time'])

            if df_shots_sh.iloc[i+1]['index 2min before'] - df_shots_sh.iloc[i]['index 2min after'] > 8*60*25:
                dif = df_shots_sh.iloc[i+1]['index 2min before'] - df_shots_sh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int(dif/2)].max()]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int(dif/3)].max()]['Time'])
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int((dif/3)*2)].max()]['Time'])
                elif dif >= 16*60*25:
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int(dif/4)].max()]['Time'])
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*2)].max()]['Time'])
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*3)].max()]['Time'])

        except:
            if sh.iloc[-1].name - df_shots_sh.iloc[i]['index 2min after'] > 8*60*25:
                dif = sh.iloc[-1].name - df_shots_sh.iloc[i]['index 2min after']
                if dif < 12*60*25:
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int(dif/2)].max()]['Time'])
                elif dif >= 12*60*25 and dif <16*60*25:
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int(dif/3)].max()]['Time'])
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int((dif/3)*2)].max()]['Time'])
                elif dif >= 16*60*25:
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int(dif/4)].max()]['Time'])
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*2)].max()]['Time'])
                    minutes_second_half.append(sh.loc[sh.index[sh.index <= df_shots_sh.iloc[i]['index 2min after'] + int((dif/4)*3)].max()]['Time'])

    minutes_second_half = [i + 45 for i in minutes_second_half]
    
    pcbhome = []
    pcbaway = []
    pcvbhome = []
    pcvbaway = []
    pcahome = []
    pcaaway = []
    pcvahome = []
    pcvaaway = []
    deltamompchome = []
    deltamompcaway = []
    deltamompcvhome = []
    deltamompcvaway = []
    deltapchome = []
    deltapcaway = []
    deltapcvhome = []
    deltapcvaway = []
    varpchome = []
    varpcaway = []
    varpcvhome = []
    varpcvaway = []
    index_min_below = []
    index_min_above = []
    minutes = []
    half = []
    seconds = []
    
    #for the new time windows: add the variables used in the regression
    #these are the same calculations as before
    for i in range(len(minutes_first_half)):
        minute = float(int(minutes_first_half[i]))
        second = float((minutes_first_half[i]-int(minutes_first_half[i]))*60)

        try:
            index_above = fh[fh['Time'] > float((minute+2) + second/60)].iloc[0].name
        except:
            index_above = fh[fh['Time'] < float((minute+2) + second/60)].iloc[-1].name
        try:
            index_below = fh[fh['Time'] <= float((minute+2) + second/60)].iloc[-1].name
        except:
            index_below = index_above
        
        index_2min_above = fh.index[fh.index <= index_above + 2*60*25].max()
        index_4min_above = fh.index[fh.index <= index_above + 4*60*25].max()
        index_2min_below = fh.index[fh.index >= index_below - 2*60*25].min()
        df_i_pc_b = fh.loc[index_2min_below:index_below]
        df_i_pc_a = fh.loc[index_2min_above:index_4min_above]
        pcbhome.append(df_i_pc_b['momentum pc home previous 2 minutes'].mean())
        pcbaway.append(df_i_pc_b['momentum pc away previous 2 minutes'].mean())
        pcvbhome.append(df_i_pc_b['momentum pcv home previous 2 minutes'].mean())
        pcvbaway.append(df_i_pc_b['momentum pcv away previous 2 minutes'].mean())
        pcahome.append(df_i_pc_a['momentum pc home previous 2 minutes'].mean())
        pcaaway.append(df_i_pc_a['momentum pc away previous 2 minutes'].mean())
        pcvahome.append(df_i_pc_a['momentum pcv home previous 2 minutes'].mean())
        pcvaaway.append(df_i_pc_a['momentum pcv away previous 2 minutes'].mean())
        deltamompchome.append(pcahome[-1] - pcbhome[-1])
        deltamompcaway.append(pcaaway[-1] - pcbaway[-1])
        deltamompcvhome.append(pcvahome[-1] - pcvbhome[-1])
        deltamompcvaway.append(pcvaaway[-1] - pcvbaway[-1])
        deltapchome.append(df_i_pc_b.iloc[-1]['pc home'] - df_i_pc_b.iloc[0]['pc home'])
        deltapcaway.append(df_i_pc_b.iloc[-1]['pc away'] - df_i_pc_b.iloc[0]['pc away'])
        deltapcvhome.append(df_i_pc_b.iloc[-1]['pcv home'] - df_i_pc_b.iloc[0]['pcv home'])
        deltapcvaway.append(df_i_pc_b.iloc[-1]['pcv away'] - df_i_pc_b.iloc[0]['pcv away'])
        varpchome.append(sum(abs(df_i_pc_b['pc home'].diff()[1:])))
        varpcaway.append(sum(abs(df_i_pc_b['pc away'].diff()[1:])))
        varpcvhome.append(sum(abs(df_i_pc_b['pcv home'].diff()[1:])))
        varpcvaway.append(sum(abs(df_i_pc_b['pcv away'].diff()[1:])))
        index_min_below.append(index_2min_below)
        index_min_above.append(index_2min_above)
        minutes.append(minute)
        seconds.append(int(second))
        half.append(1)
        
    for i in range(len(minutes_second_half)):
        minute = float(int(minutes_second_half[i])-45)
        second = float((minutes_second_half[i]-int(minutes_second_half[i]))*60)
        try:
            index_above = sh[sh['Time'] > float((minute-43) + second/60)].iloc[0].name
        except:
            index_above = sh[sh['Time'] < float((minute-43) + second/60)].iloc[-1].name
        try:
            index_below = sh[sh['Time'] <= float((minute-43) + second/60)].iloc[-1].name
        except:
            index_below = index_above
        
        index_2min_above = sh.index[sh.index <= index_above + 2*60*25].max()
        index_4min_above = sh.index[sh.index <= index_above + 4*60*25].max()
        index_2min_below = sh.index[sh.index >= index_below - 2*60*25].min()
        df_i_pc_b = sh.loc[index_2min_below:index_below]
        df_i_pc_a = sh.loc[index_2min_above:index_4min_above]
        pcbhome.append(df_i_pc_b['momentum pc home previous 2 minutes'].mean())
        pcbaway.append(df_i_pc_b['momentum pc away previous 2 minutes'].mean())
        pcvbhome.append(df_i_pc_b['momentum pcv home previous 2 minutes'].mean())
        pcvbaway.append(df_i_pc_b['momentum pcv away previous 2 minutes'].mean())
        pcahome.append(df_i_pc_a['momentum pc home previous 2 minutes'].mean())
        pcaaway.append(df_i_pc_a['momentum pc away previous 2 minutes'].mean())
        pcvahome.append(df_i_pc_a['momentum pcv home previous 2 minutes'].mean())
        pcvaaway.append(df_i_pc_a['momentum pcv away previous 2 minutes'].mean())
        deltamompchome.append(pcahome[-1] - pcbhome[-1])
        deltamompcaway.append(pcaaway[-1] - pcbaway[-1])
        deltamompcvhome.append(pcvahome[-1] - pcvbhome[-1])
        deltamompcvaway.append(pcvaaway[-1] - pcvbaway[-1])
        deltapchome.append(df_i_pc_b.iloc[-1]['pc home'] - df_i_pc_b.iloc[0]['pc home'])
        deltapcaway.append(df_i_pc_b.iloc[-1]['pc away'] - df_i_pc_b.iloc[0]['pc away'])
        deltapcvhome.append(df_i_pc_b.iloc[-1]['pcv home'] - df_i_pc_b.iloc[0]['pcv home'])
        deltapcvaway.append(df_i_pc_b.iloc[-1]['pcv away'] - df_i_pc_b.iloc[0]['pcv away'])
        varpchome.append(sum(abs(df_i_pc_b['pc home'].diff()[1:])))
        varpcaway.append(sum(abs(df_i_pc_b['pc away'].diff()[1:])))
        varpcvhome.append(sum(abs(df_i_pc_b['pcv home'].diff()[1:])))
        varpcvaway.append(sum(abs(df_i_pc_b['pcv away'].diff()[1:])))
        index_min_below.append(index_2min_below)
        index_min_above.append(index_2min_above)
        minutes.append(minute + 45)
        seconds.append(int(second))
        half.append(2)
        
    index = []
    type_id = []
    type_name = []
    team_id = []
    frame_id = []
    team = []
    outcome = []
    match_id = []
    key_moment = []
    type_key = []

    #also add the other data in the original dataframe for the new time windows
    for i in range(len(half)):
        index.append(df_shots.iloc[-1].name + i)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        outcome.append(0)
        match_id.append(df_shots.iloc[0]['match_id'])
        key_moment.append('neutral')
        type_key.append(0)
        frame_id.append(0)

    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minutes,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
           'outcome': outcome,
           'match_id': match_id,
           'key_moment': key_moment,
            'type_key': type_key,
            'mean pc home 2min before': pcbhome,
            'mean pc away 2min before': pcbaway,
            'mean pc home 2min after': pcahome,
            'mean pc away 2min after': pcaaway,
            'mean pcv home 2min before': pcvbhome,
            'mean pcv away 2min before': pcvbaway,
            'mean pcv home 2min after': pcvahome,
            'mean pcv away 2min after': pcvaaway,
            'delta pc momentum home': deltamompchome,
            'delta pc momentum away': deltamompcaway,
            'delta pcv momentum home': deltamompcvhome,
            'delta pcv momentum away': deltamompcvaway,
            'end value pc home': endpchome,
            'end value pc away': endpcaway,
            'end value pcv home': endpcvhome,
            'end value pcv away': endpcvaway,
            'delta pc home': deltapchome,
            'delta pc away': deltapcaway,
            'delta pcv home': deltapcvhome,
            'delta pcv away': deltapcvaway,
            'var pc home': varpchome,
            'var pc away': varpcaway,
            'var pcv home': varpcvhome,
            'var pcv away': varpcvaway,
           'index 2min before': index_min_below,
           'index 2min after': index_min_above}
    
    #create a dataframe of the new time windows
    df = pd.DataFrame(data)
    #add this dataframe to the original shots and goals dataframe for each match
    df_new = pd.concat([df_shots, df])
    #add this to the dataframe for all matches
    final_df = pd.concat([final_df, df_new])
#export the complete dataframe to a pickle file that can be used for regression
pickle.dump(final_df, open('input PC regression key moments on Momentum.p', 'wb')) 





In [ ]:
#Calculations that will be used for regression input for key moments on the flow of momentum (xT calculations)
from progressbar import ProgressBar
import pickle
import pandas as pd

#import the shots and goals dataframe
infile = open('shots and goals event dataframes.p','rb')
df_shot = pickle.load(infile)
infile.close()

#import the dataframe including xT in sliding time windows for all mathes
infile = open('all matches sliding windows xT.p','rb')
df_xT = pickle.load(infile)
infile.close()
maximum = max(df_xT['xT_home'].max(), df_xT['xT_away'].max())
minimum = min(df_xT['xT_home'].min(), df_xT['xT_away'].min())

#normalize the values
df_xT['xT_home'] = (df_xT['xT_home']-minimum)/(maximum-minimum)
df_xT['xT_away'] = (df_xT['xT_away']-minimum)/(maximum-minimum)
df_xT['xT_prev_2min_home'] = (df_xT['xT_prev_2min_home']-minimum)/(maximum-minimum)
df_xT['xT_prev_2min_away'] = (df_xT['xT_prev_2min_away']-minimum)/(maximum-minimum)
#create an array with all the different matches in the xT dataframe
matches = df_xT.Match.unique()
pbar = ProgressBar()
#create empty dataframe
final_df = pd.DataFrame()

#loop over each match in the array
for game in pbar(matches):
    
    #filter the shots and goals dataframe and the xt dataframe on each match
    df_shots = df_shot.loc[df_shot['match_id'] == game]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    #create separate dataframes for the first and second half
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    minutes_first_half = []
    
    #for each shot and goal in the dataframe: add whether it is favorable or unfavorable for the home team
    type_key = []
    key_mom = []
    for i in range(len(df_shots)):
        if df_shots.iloc[i]['team'] == 'Home':
            key_mom.append('favorable')
            type_key.append(1)
        else:
            key_mom.append('unfavorable')
            type_key.append(-1)
    
    df_shots['key_moment'] = key_mom
    df_shots['type_key'] = type_key
    
    #check whether there are time windows in which no key moments occurs. If yes, add the minutes to a list
    #the calculations are the same as used in the PC and PCV model. Only this model considers regular playing time
    for i in range(len(df_shots_fh)):
        try:
            if i == 0:
                if int(df_shots_fh.iloc[i]['minute']) + int(df_shots_fh.iloc[i]['seconds'])/60  > 8:
                    dif = int(df_shots_fh.iloc[i]['minute']) + int(df_shots_fh.iloc[i]['seconds'])/60
                    if dif < 12:
                        minutes_first_half.append(dif/2)
                    elif dif >= 12 and dif <16:
                        minutes_first_half.append(dif/3)
                        minutes_first_half.append(2*dif/3)
                    elif dif >= 16:
                        minutes_first_half.append(1*dif/4)
                        minutes_first_half.append(2*dif/4)
                        minutes_first_half.append(3*dif/4)

            if (int(df_shots_fh.iloc[i+1]['minute'])+int(df_shots_fh.iloc[i+1]['seconds'])/60) - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60) > 8:                
                dif = (int(df_shots_fh.iloc[i+1]['minute'])+int(df_shots_fh.iloc[i+1]['seconds'])/60) - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 3*dif/4)

        except:
            if 46 - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60) > 8:
                dif = 46 - (int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 1*dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_first_half.append(int(df_shots_fh.iloc[i]['minute'])+int(df_shots_fh.iloc[i]['seconds'])/60 + 3*dif/4)


    #do the same for the second half
    minutes_second_half = []
    for i in range(len(df_shots_sh)):
        try:
            if i == 0:
                if (int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60) - 45  > 8:
                    dif = (int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60) - 45
                    if dif < 12:
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - dif/2)
                    elif dif >= 12 and dif <16:
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - dif/3)
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 2*dif/3)
                    elif dif >= 16:
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 1*dif/4)
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 2*dif/4)
                        minutes_second_half.append(int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60 - 3*dif/4)

            if (int(df_shots_sh.iloc[i+1]['minute'])+int(df_shots_sh.iloc[i+1]['seconds'])/60) - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60) > 8:
                dif = (int(df_shots_sh.iloc[i+1]['minute'])+int(df_shots_sh.iloc[i+1]['seconds'])/60) - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 3*dif/4)

        except:
            if 91 - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60) > 8:
                dif = 91 - (int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60)
                if dif < 12:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/2)
                elif dif >= 12 and dif <16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + dif/3)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/3)
                elif dif >= 16:
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 1*dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 2*dif/4)
                    minutes_second_half.append(int(df_shots_sh.iloc[i]['minute'])+int(df_shots_sh.iloc[i]['seconds'])/60 + 3*dif/4)
        
    df_shots = df_shots[['index', 'type_id', 'type_name', 'half', 'frame_id', 'minute', 'seconds', 'team_id', 'team', 'outcome', 'match_id', 'key_moment', 'type_key']]

    index = []
    type_id = []
    type_name = []
    team_id = []
    frame_id = []
    team = []
    outcome = []
    match_id = []
    key_moment = []
    type_key = []
    half = []
    minute = []
    seconds = []

    #for the new time windows, add the other data 
    for j in range(len(minutes_first_half)):
        index.append(df_shots_fh.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        outcome.append(0)
        match_id.append(df_shots_fh.iloc[0]['match_id'])
        key_moment.append('neutral')
        type_key.append(0)
        frame_id.append(0)
        half.append('1')
        minute.append(int(minutes_first_half[j]))
        seconds.append((minutes_first_half[j] - int(minutes_first_half[j]))*60)

    for j in range(len(minutes_second_half)):
        index.append(df_shots_sh.iloc[-1].name +j)
        type_id.append(0)
        type_name.append('No')
        team_id.append(0)
        team.append('Neutral')
        outcome.append(0)
        match_id.append(df_shots_sh.iloc[0]['match_id'])
        key_moment.append('neutral')
        type_key.append(0)
        frame_id.append(0)
        half.append('2')
        minute.append(int(minutes_second_half[j]))
        seconds.append((minutes_second_half[j] - int(minutes_second_half[j]))*60)

    data = {'index': index,
           'type_id': type_id,
           'type_name': type_name,
           'half': half,
           'frame_id': frame_id,
           'minute': minute,
           'seconds': seconds,
           'team_id': team_id,
           'team': team,
           'outcome': outcome,
           'match_id': match_id,
           'key_moment': key_moment,
            'type_key': type_key}

    #create a dataframe for the new time windows
    df = pd.DataFrame(data)
    #add this dataframe to the shots and goals dataframe of that match
    df_new = pd.concat([df_shots, df])
    #add this complete match dataframe to the dataframe for all matches
    #this dataframe is sill without xT values
    final_df = pd.concat([final_df, df_new])
        
            


In [ ]:
pbar = ProgressBar()
#create empty dataframe
df_final = pd.DataFrame()
#iterate over all matches
for game in pbar(matches):
    #again split on first and second half
    df_shots = final_df.loc[final_df['match_id'] == game]
    df_match_xT = df_xT.loc[df_xT['Match'] == game]
    df_xT_fh = df_match_xT.loc[df_match_xT['Half'] == 1]
    df_xT_sh = df_match_xT.loc[df_match_xT['Half'] == 2]
    df_shots_fh = df_shots.loc[df_shots['half'] == '1']
    df_shots_sh = df_shots.loc[df_shots['half'] == '2']
    
    #initialize the necessary lists
    xthomeb = []
    xtawayb = []
    xthomea = []
    xtawaya = []
    mxthomeb = []
    mxtawayb = []
    mxthomea = []
    mxtawaya = []
    deltamomxthome = []
    deltamomxtaway = []
    deltaxthome = []
    deltaxtaway = []
    varxthome = []
    varxtaway = []
    
    
    #for each row in the shots and goals dataframe: add the xt values for the time windows
    #These calculations are again approximately the same as in the PC and PCV models
    for i in range(len(df_shots_fh)):
        time = int(df_shots_fh.iloc[i]['minute']) + int(df_shots_fh.iloc[i]['seconds'])/60
        try:
            df_i_xtb = df_xT_fh.loc[df_xT_fh.index[df_xT_fh['Minute'] <= time - 2][-1]:df_xT_fh.index[df_xT_fh['Minute'] <= time][-1]]
        except:
            df_i_xtb = df_xT_fh.loc[df_xT_fh.index[df_xT_fh['Minute'] >= time - 2][0]:df_xT_fh.index[df_xT_fh['Minute'] <= time][-1]]
        df_i_xta = df_xT_fh.loc[df_xT_fh.index[df_xT_fh['Minute'] <= time + 2][-1]:df_xT_fh.index[df_xT_fh['Minute'] <= time + 4][-1]]
        xthomeb.append(df_i_xtb['xT_prev_2min_home'].mean())
        xtawayb.append(df_i_xtb['xT_prev_2min_away'].mean())
        xthomea.append(df_i_xta['xT_prev_2min_home'].mean())
        xtawaya.append(df_i_xta['xT_prev_2min_away'].mean())
        deltamomxthome.append(xthomea[-1]-xthomeb[-1])
        deltamomxtaway.append(xtawaya[-1]-xtawayb[-1])
        df_inter = df_xT_fh.loc[df_xT_fh['Minute'] <= time]
        if len(df_inter) == 0:
            df_inter = df_xT_fh.loc[df_xT_fh['Minute'] >= time].iloc[0]
            df_delta = df_inter
        elif len(df_inter) == 1:
            df_delta = df_inter
        else:
            df_delta = df_inter.iloc[-min(5, len(df_inter)): -1]
        deltaxthome.append(df_delta.iloc[-1]['xT_home'] - df_delta.iloc[0]['xT_home'])
        deltaxtaway.append(df_delta.iloc[-1]['xT_away'] - df_delta.iloc[0]['xT_away'])
        varxthome.append(sum(abs(df_delta['xT_home'].diff()[1:])))
        varxtaway.append(sum(abs(df_delta['xT_away'].diff()[1:])))
        

    #do the same for the second half
    for i in range(len(df_shots_sh)):
        time = int(df_shots_sh.iloc[i]['minute']) + int(df_shots_sh.iloc[i]['seconds'])/60
        try:
            df_i_xtb = df_xT_sh.loc[df_xT_sh.index[df_xT_sh['Minute'] <= time - 2][-1]:df_xT_sh.index[df_xT_sh['Minute'] <= time][-1]]
        except:
            df_i_xtb = df_xT_sh.loc[df_xT_sh.index[df_xT_sh['Minute'] >= time - 2][0]:df_xT_sh.index[df_xT_sh['Minute'] <= time][-1]]
        df_i_xta = df_xT_sh.loc[df_xT_sh.index[df_xT_sh['Minute'] <= time + 2][-1]:df_xT_sh.index[df_xT_sh['Minute'] <= time + 4][-1]]
        xthomeb.append(df_i_xtb['xT_prev_2min_home'].mean())
        xtawayb.append(df_i_xtb['xT_prev_2min_away'].mean())
        xthomea.append(df_i_xta['xT_prev_2min_home'].mean())
        xtawaya.append(df_i_xta['xT_prev_2min_away'].mean())
        deltamomxthome.append(xthomea[-1]-xthomeb[-1])
        deltamomxtaway.append(xtawaya[-1]-xtawayb[-1])
        df_inter = df_xT_sh.loc[df_xT_sh['Minute'] <= time]
        if len(df_inter) == 0:
            df_inter = df_xT_sh.loc[df_xT_sh['Minute'] >= time].iloc[0]
            df_delta = df_inter
        elif len(df_inter) == 1:
            df_delta = df_inter
        else:
            df_delta = df_inter.iloc[-min(5, len(df_inter)): -1]
        deltaxthome.append(df_delta.iloc[-1]['xT_prev_2min_home'] - df_delta.iloc[0]['xT_prev_2min_home'])
        deltaxtaway.append(df_delta.iloc[-1]['xT_prev_2min_away'] - df_delta.iloc[0]['xT_prev_2min_away'])
        varxthome.append(sum(abs(df_delta['xT_home'].diff()[1:])))
        varxtaway.append(sum(abs(df_delta['xT_away'].diff()[1:])))
    
    df_shots = pd.concat([df_shots_fh, df_shots_sh])
        
    
    df_shots['mean xt home 2min before'] = xthomeb
    df_shots['mean xt away 2min before'] = xtawayb
    df_shots['mean xt home 2min after'] = xthomea
    df_shots['mean xt away 2min after'] = xtawaya
    df_shots['delta mom home'] = deltamomxthome
    df_shots['delta mom away'] = deltamomxtaway
    df_shots['end xt home'] = endxthome
    df_shots['end xt away'] = endxtaway
    df_shots['delta xt home before'] = deltaxthome
    df_shots['delta xt away before'] = deltaxtaway
    df_shots['var xt home before'] = varxthome
    df_shots['var xt away before'] = varxtaway
    
    #create a complete dataframe for all matches with the xT values
    df_final = pd.concat([df_shots, df_final])

#export the dataframe for all the matches to a pickle file that can be used for regression
pickle.dump(df_final, open('input xT regression key moments on Momentum.p', 'wb'))    